# Проект по A/B тестированию

## Цели и задачи

**Цель:** провести оценку результатов A/B-теста

**Задачи:**
1. Оценить корректность проведения теста.
2. Проанализировать результаты теста. Для этого необходимо проверить 1) пересечение тестовой аудитории с конкурирующим тестом; 2) совпадение теста и маркетинговых событий, другие проблемы временных границ теста. 

## Прочтение и знакомство с данными

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as stats
import numpy as np
import seaborn as sns

In [16]:
calendar = pd.read_csv('/datasets/ab_project_marketing_events.csv')
all_users = pd.read_csv('/datasets/final_ab_new_users.csv')
all_events = pd.read_csv('/datasets/final_ab_events.csv')
test_users = pd.read_csv('/datasets/final_ab_participants.csv')

In [4]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [5]:
calendar.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


In [12]:
calendar['start_dt'] = calendar['start_dt'].astype('datetime64[D]')
calendar['finish_dt'] = calendar['finish_dt'].astype('datetime64[D]')

In [6]:
all_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [7]:
all_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [13]:
all_users['first_date'] = all_users['first_date'].astype('datetime64[D]')

In [8]:
all_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [9]:
all_events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


In [14]:
all_events['event_dt'] = all_events['event_dt'].astype('datetime64[D]')

In [17]:
test_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [18]:
test_users.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [15]:
test_users['event_dt'] = test_users['event_dt'].astype('datetime64[D]')

При первом знакомстве с данными, сразу видно необходимость изменить типы переменных с датами во всех датасетах

## Предобработка данных

### Пропуски

Пропущенные значения замечены в датасете all_events по столбцу details, который обозначает дополнительные данные о событии. Проверим, с чем они связаны

In [21]:
all_events.pivot_table(values='details', index='event_name', aggfunc=('count'))

,details
event_name,
login,0
product_cart,0
product_page,0
purchase,62740


Пропущенные значения связаны с типом события. Детали в виде стоимости покупок хранятся лишь по событию "purchase".

In [25]:
all_events[all_events['event_name'] == 'purchase'].count()

user_id       62740
event_dt      62740
event_name    62740
details       62740
dtype: int64

В самом событии "purchase" пропущенных значений нет. С остальными пропусками ничего делать не нужно.

### Дубликаты

In [28]:
duplicateRows = calendar[calendar.duplicated ()]
duplicateRows

,name,regions,start_dt,finish_dt


In [29]:
duplicateRows = all_users[all_users.duplicated ()]
duplicateRows

,user_id,first_date,region,device


In [30]:
duplicateRows = all_events[all_events.duplicated ()]
duplicateRows

,user_id,event_dt,event_name,details


In [31]:
duplicateRows = test_users[test_users.duplicated ()]
duplicateRows

,user_id,group,ab_test


Дубликаты в данных не обнаружены